### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import pandas as pd
import os
import csv

# File to Load (Remember to Change These if my path is different)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [6]:
school_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [7]:
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [16]:
student_data["student_name"].count

<bound method Series.count of 0             Paul Bradley
1             Victor Smith
2          Kevin Rodriguez
3        Dr. Richard Scott
4               Bonnie Ray
               ...        
39165         Donna Howard
39166            Dawn Bell
39167       Rebecca Tanner
39168         Desiree Kidd
39169      Carolyn Jackson
Name: student_name, Length: 39170, dtype: object>

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing), **count of unique student name from each list/all students

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [17]:
total_schools = len(school_data["School ID"])

total_students = len(student_data["Student ID"])

total_budget = school_data["budget"].sum()

avg_mathscore = student_data["math_score"].mean()

avg_readingscore = student_data["reading_score"].mean()

students_passmath = student_data.loc[student_data["math_score"] >= 70]["math_score"].count()
percent_passmath = students_passmath/total_students

students_passreading = student_data.loc[student_data["reading_score"] >= 70]["reading_score"].count()
percent_passreading = students_passreading/total_students

overall_passing = student_data[(student_data["math_score"] >= 70) & (student_data["reading_score"] >= 70)]["Student ID"].count()/total_students

district_summary = pd.DataFrame({"Total Schools": [total_schools],"Total Students": [total_students],"Total School Budget": [total_budget],
                                 "Average Math Score": [avg_mathscore],"Average Reading Score": [avg_readingscore],
                                 "% Passing Math": [percent_passmath],"% Passing Reading": [percent_passreading],
                                 "% Overall Passing": [overall_passing]})

district_summary.style.format({"Total Students": "{:,}", "Total School Budget": "${:,.2f}", "Average Math Score": "{:.2f}", "Average Reading Score": "{:.2f}", 
                               "% Passing Math": "{:.2%}", "% Passing Reading": "{:.2%}", 
                               "% Overall Passing": "{:.2%}"})

,Total Schools,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [18]:
per_school = school_data_complete.set_index("school_name").groupby(["school_name"])

school_types = school_data.set_index("school_name")["type"]

students_per = per_school["Student ID"].count()

school_budget = school_data.set_index("school_name")["budget"]

student_budget = school_data.set_index("school_name")["budget"]/school_data.set_index("school_name")["size"]

avg_schoolmath = per_school["math_score"].mean()
avg_schoolread = per_school["reading_score"].mean()

pass_schoolmath = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name")["Student ID"].count()/students_per 
pass_schoolread = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name")["Student ID"].count()/students_per 
overallschool = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("school_name")["Student ID"].count()/students_per 

school_summary = pd.DataFrame({"School Type": school_types,"Total Students": students_per,"Total School Budget": school_budget,
                               "Per Student Budget": student_budget,"Average Math Score": avg_schoolmath,"Average Reading Score": avg_schoolread,
                               "% Passing Math": pass_schoolmath,"% Passing Reading": pass_schoolread,"% Overall Passing": overallschool})

school_summary.style.format({"Total Students": "{:,}", "Total School Budget": "${:,}", "Per Student Budget": "${:.0f}",
                             "Average Math Score": "{:.2f}", "Average Reading Score": "{:.2f}", "% Passing Math": "{:.2%}",
                             "% Passing Reading": "{:.2%}", "% Overall Passing": "{:.2%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [19]:
top_five = school_summary.sort_values("% Overall Passing", ascending = False)
top_five.head().style.format({"Total Students": '{:,}', "Total School Budget": "${:,}", "Per Student Budget": "${:.0f}",
                              "Average Math Score": "{:.2f}", "Average Reading Score": "{:.2f}", "% Passing Math": "{:.2%}",
                              "% Passing Reading": "{:.2%}", "% Overall Passing": "{:.2%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [20]:
bottom_five = school_summary.sort_values("% Overall Passing", ascending = True)
bottom_five.head().style.format({"Total Students": "{:,}", "Total School Budget": "${:,}", "Per Student Budget": "${:.0f}",
                              "Average Math Score": "{:.2f}", "Average Reading Score": "{:.2f}", "% Passing Math": "{:.2%}",
                              "% Passing Reading": "{:.2%}", "% Overall Passing": "{:.2%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Math Scores for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [21]:
math_nine = student_data.loc[student_data["grade"] == "9th"].groupby("school_name")["math_score"].mean()
math_ten = student_data.loc[student_data["grade"] == "10th"].groupby("school_name")["math_score"].mean()
math_eleven = student_data.loc[student_data["grade"] == "11th"].groupby("school_name")["math_score"].mean()
math_twelve = student_data.loc[student_data["grade"] == "12th"].groupby("school_name")["math_score"].mean()

math_scores = pd.DataFrame({"9th": math_nine, "10th": math_ten, "11th": math_eleven, "12th": math_twelve})
math_scores = math_scores[["9th", "10th", "11th", "12th"]]
math_scores.index.name = ""

math_scores.style.format({"9th": "{:.2f}", "10th": "{:.2f}", "11th": "{:.2f}", "12th": "{:.2f}"})

,9th,10th,11th,12th
,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [22]:
reading_nine = student_data.loc[student_data["grade"] == "9th"].groupby("school_name")["reading_score"].mean()
reading_ten = student_data.loc[student_data["grade"] == "10th"].groupby("school_name")["reading_score"].mean()
reading_eleven = student_data.loc[student_data["grade"] == "11th"].groupby("school_name")["reading_score"].mean()
reading_twelve = student_data.loc[student_data["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

reading_scores = pd.DataFrame({"9th": reading_nine, "10th": reading_ten, "11th": reading_eleven, "12th": reading_twelve})
reading_scores = reading_scores[["9th", "10th", "11th", "12th"]]
reading_scores.index.name = ""

reading_scores.style.format({"9th": "{:.2f}", "10th": "{:.2f}", "11th": "{:.2f}", "12th": "{:.2f}"})

,9th,10th,11th,12th
,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [23]:
bins = [0, 584.49, 629.49, 644.49, 9999999999]
group_name = ["<$584", "$585-629", "$630-644", "$645-675"]
school_data_complete["spending_bins"] = pd.cut(school_data_complete["budget"]/school_data_complete["size"], bins, labels = group_name)

by_spending = school_data_complete.groupby("spending_bins")

avg_schoolmath = by_spending["math_score"].mean()
avg_schoolread = by_spending["reading_score"].mean()
pass_schoolmath = school_data_complete[school_data_complete["math_score"] >= 70].groupby("spending_bins")["Student ID"].count()/by_spending["Student ID"].count()
pass_schoolread = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("spending_bins")["Student ID"].count()/by_spending["Student ID"].count()
overallschool = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("spending_bins")["Student ID"].count()/by_spending["Student ID"].count()

scores_by_spend = pd.DataFrame({"Average Math Score": avg_schoolmath, "Average Reading Score": avg_schoolread,
                                "% Passing Math": pass_schoolmath, "% Passing Reading": pass_schoolread, "% Overall Passing": overallschool})
            
scores_by_spend.index.name = "Spending Ranges (Per Student)"
scores_by_spend = scores_by_spend.reindex(group_name)

scores_by_spend.style.format({"Average Math Score": "{:.2f}", "Average Reading Score": "{:.2f}", "% Passing Math": "{:.2%}",
                              "% Passing Reading": "{:.2%}", "% Overall Passing": "{:.2%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.36,83.96,93.70%,96.69%,90.64%
$585-629,79.98,82.31,79.11%,88.51%,70.94%
$630-644,77.82,81.30,70.62%,82.60%,58.84%
$645-675,77.05,81.01,66.23%,81.11%,53.53%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [24]:
bins = [0, 999.49, 1999.49, 9999999999]
group_name = ["Small (<1000)", "Medium (1000-2000)" , "Large (2000-5000)"]
school_data_complete["size_bins"] = pd.cut(school_data_complete["size"], bins, labels = group_name)

by_size = school_data_complete.groupby("size_bins")

avg_schoolmath = by_size["math_score"].mean()
avg_schoolread = by_size["reading_score"].mean()
pass_schoolmath = school_data_complete[school_data_complete["math_score"] >= 70].groupby("size_bins")["Student ID"].count()/by_size["Student ID"].count()
pass_schoolread = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("size_bins")["Student ID"].count()/by_size["Student ID"].count()
overallschool = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("size_bins")["Student ID"].count()/by_size["Student ID"].count()
 
scores_by_size = pd.DataFrame({"Average Math Score": avg_schoolmath, "Average Reading Score": avg_schoolread,
                                "% Passing Math": pass_schoolmath, "% Passing Reading": pass_schoolread, "% Overall Passing": overallschool})

scores_by_size.index.name = "School Size"
scores_by_size = scores_by_size.reindex(group_name)

scores_by_size.style.format({"Average Math Score": "{:.2f}", "Average Reading Score": "{:.2f}", "% Passing Math": "{:.2%}",
                              "% Passing Reading": "{:.2%}", "% Overall Passing": "{:.2%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.83,83.97,93.95%,96.04%,90.14%
Medium (1000-2000),83.37,83.87,93.62%,96.77%,90.62%
Large (2000-5000),77.48,81.20,68.65%,82.13%,56.57%


## Scores by School Type

* Perform the same operations as above, based on school type

In [25]:
by_type = school_data_complete.groupby("type")

avg_schoolmath = by_type["math_score"].mean()
avg_schoolread = by_type["reading_score"].mean()
pass_schoolmath = school_data_complete[school_data_complete["math_score"] >= 70].groupby("type")["Student ID"].count()/by_type["Student ID"].count()
pass_schoolread = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("type")["Student ID"].count()/by_type["Student ID"].count()
overallschool = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("type")["Student ID"].count()/by_type["Student ID"].count()
          
scores_by_type = pd.DataFrame({"Average Math Score": avg_schoolmath, "Average Reading Score": avg_schoolread,
                                "% Passing Math": pass_schoolmath, "% Passing Reading": pass_schoolread, "% Overall Passing": overallschool})

scores_by_type.index.name = "School Type"

scores_by_type.style.format({"Average Math Score": "{:.2f}", "Average Reading Score": "{:.2f}", "% Passing Math": "{:.2%}",
                              "% Passing Reading": "{:.2%}", "% Overall Passing": "{:.2%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.41,83.90,93.70%,96.65%,90.56%
District,76.99,80.96,66.52%,80.91%,53.70%
